In [0]:
print('test')

test	


In [0]:
print("hello form lua and torch")

hello form lua and torch	


In [0]:
require 'nngraph';

{
  setDebug : function: 0x022838e8
  Node : table: 0x021ab480
  JustTable : table: 0x02288028
  annotateNodes : function: 0x022839d8
  nest : function: 0x021a4c90
  JustElement : table: 0x02286e88
  simple_print : 
    {
      todot : function: 0x0228bf50
      dot : function: 0x0228bfc8
    }
  display : function: 0x022839f8
}


In [0]:
-- simple feed forward neural network easily
-- common style to mark inputs with identity nodes for clarity
input = nn.Identity()()

In [0]:
-- each hidden layer is achieved by connecting the previous one
-- here we define a single hidden layer network
h1 = nn.Tanh()(nn.Linear(20, 10)(input))
output = nn.Linear(10, 1)(h1)
mlp = nn.gModule({input}, {output})

x = torch.rand(20)
dx = torch.rand(1)

mlp:updateOutput(x)
mlp:updateGradInput(x, dx)
mlp:accGradParameters(x, dx)

/Users/jm/torch/install/share/lua/5.1/nngraph/gmodule.lua:124: Expected nnop.Parameters node, module not found in node
stack traceback:
	[C]: in function 'assert'
	/Users/jm/torch/install/share/lua/5.1/nngraph/gmodule.lua:124: in function '__init'
	/Users/jm/torch/install/share/lua/5.1/torch/init.lua:91: in function </Users/jm/torch/install/share/lua/5.1/torch/init.lua:87>
	[C]: in function 'gModule'
	[string "-- each hidden layer is achieved by connectin..."]:5: in main chunk
	[C]: in function 'xpcall'
	/Users/jm/torch/install/share/lua/5.1/itorch/main.lua:209: in function </Users/jm/torch/install/share/lua/5.1/itorch/main.lua:173>
	/Users/jm/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/jm/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/jm/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/jm/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/jm/torch/install/share

In [0]:
-- draw graph (the forward graph, '.fg')
-- thsi will produce an SVG in the runtime directory
graph.dot(mpl.fg, 'MLP', 'MLP')
itorch.image('MPL.svg')

[string "-- draw graph (the forward graph, '.fg')..."]:3: attempt to index global 'mpl' (a nil value)
stack traceback:
	[string "-- draw graph (the forward graph, '.fg')..."]:3: in main chunk
	[C]: in function 'xpcall'
	/Users/jm/torch/install/share/lua/5.1/itorch/main.lua:209: in function </Users/jm/torch/install/share/lua/5.1/itorch/main.lua:173>
	/Users/jm/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/jm/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/jm/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/jm/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/jm/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x01014edd20: 

In [0]:
local function get_network()
    -- it is common style to mark inputs with identity nodes for clarity.
    local input = nn.Identity()()

    -- each hidden layer is achieved by connecting the previous one
    -- here we define a single hidden layer network
    local h1 = nn.Linear(20, 10)(input)
    local h2 = nn.Sigmoid()(h1)
    local output = nn.Linear(10, 1)(h2)
    
    -- the following function call inspects the local variables in this
    -- function and finds the nodes corresponding to local variables.
    nngraph.annotateNodes()
    return nn.gModule({input}, {output}) 
end
mlp = get_network()
x = torch.rand(20)
dx = torch.rand(1)
mlp:updateOutput(x)
mlp:updateGradInput(x, dx)
mlp:accGradParameters(x, dx)

-- draw graph (the forward graph, '.fg')
-- this will produce an SVG in the runtime directory
graph.dot(mlp.fg, 'MLP', 'MLP_Annotated')
itorch.image('MLP_Annotated.svg')

In [0]:
-- We need to set debug flag to true
nngraph.setDebug(true)

local function get_network()
    -- it is common style to mark inputs with identity nodes for clarity.
    local input = nn.Identity()()

    -- each hidden layer is achieved by connecting the previous one
    -- here we define a single hidden layer network
    local h1 = nn.Linear(20, 10)(input)
    local h2 = nn.Sigmoid()(h1)
    local output = nn.Linear(10, 1)(h2)
    
    -- the following function call inspects the local variables in this
    -- function and finds the nodes corresponding to local variables.
    nngraph.annotateNodes()
    return nn.gModule({input}, {output}) 
end
mlp = get_network()
mlp.name = 'MyMLPError'
x = torch.rand(15) -- note that this input will cause runtime error

-- We do protected call to avoid real error interrupting the notebook
local o, err = pcall(function() mlp:updateOutput(x) end)
itorch.image('MyMLPError.svg')

In [0]:
function get_rnn(input_size, rnn_size)
  
    -- there are n+1 inputs (hiddens on each layer and x)
    local input = nn.Identity()()
    local prev_h = nn.Identity()()

    -- RNN tick
    local i2h = nn.Linear(input_size, rnn_size)(input)
    local h2h = nn.Linear(rnn_size, rnn_size)(prev_h)
    local added_h = nn.CAddTable()({i2h, h2h})
    local next_h = nn.Tanh()(added_h)
    
    nngraph.annotateNodes()
    return nn.gModule({input, prev_h}, {next_h})
end

local rnn_net = get_rnn(128, 128)
graph.dot(rnn_net.fg, 'rnn_net', 'rnn_net')
itorch.image('rnn_net.svg'

[string "function get_rnn(input_size, rnn_size)..."]:20: ')' expected (to close '(' at line 19) near '<eof>': 

In [0]:
-- connect RNN\'s in time
local function get_rnn2(input_size, rnn_size)
    local input1 = nn.Identity()()
    local input2 = nn.Identity()()
    local prev_h = nn.Identity()()
    local rnn_net1 = get_rnn(128, 128)({input1, prev_h})
    local rnn_net2 = get_rnn(128, 128)({input2, rnn_net1})
    nngraph.annotateNodes()
    return nn.gModule({input1, input2, prev_h}, {rnn_net2})
end
local rnn_net2 = get_rnn2(128, 128)
graph.dot(rnn_net2.fg, 'rnn_net2', 'rnn_net2')
itorch.image('rnn_net2.svg')

[string "-- connect RNN\'s in time..."]:6: attempt to call global 'get_rnn' (a nil value)
stack traceback:
	[string "-- connect RNN\'s in time..."]:6: in function 'get_rnn2'
	[string "-- connect RNN\'s in time..."]:11: in main chunk
	[C]: in function 'xpcall'
	/Users/jm/torch/install/share/lua/5.1/itorch/main.lua:209: in function </Users/jm/torch/install/share/lua/5.1/itorch/main.lua:173>
	/Users/jm/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/jm/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/jm/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/jm/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/jm/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x01014edd20: 

In [0]:
-- Even with variable naming, the graph becomes very complicated very
quickly. One can then use custom annotations to mark certain paths.

local function get_rnn2(input_size, rnn_size)
    local input1 = nn.Identity()():annotate{graphAttributes = {style='filled', fillcolor='blue'}}
    local input2 = nn.Identity()():annotate{graphAttributes = {style='filled', fillcolor='blue'}}
    local prev_h = nn.Identity()():annotate{graphAttributes = {style='filled', fillcolor='blue'}}
    local rnn_net1 = get_rnn(128, 128)({input1, prev_h}):annotate{graphAttributes = {style='filled', fillcolor='yellow'}}
    local rnn_net2 = get_rnn(128, 128)({input2, rnn_net1}):annotate{graphAttributes = {style='filled', fillcolor='green'}}
    nngraph.annotateNodes()
    return nn.gModule({input1, input2, prev_h}, {rnn_net2})
end
local rnn_net3 = get_rnn2(128, 128)
graph.dot(rnn_net3.fg, 'rnn_net3', 'rnn_net3')
itorch.image('rnn_net3.svg')

[string "-- Even with variable naming, the graph becom..."]:2: '=' expected near 'can': 